# Azure AI Foundry Agents with Azure AI Search

Welcome! This notebook will walk you through creating an AI agent that can search and answer questions about products in the **zava-products** search index using Azure AI Foundry Agent Service.

## What You'll Learn
- How to authenticate with Azure
- How to configure Azure AI Search tool for an agent
- How to create an AI agent with search capabilities
- How to ask questions about product data
- How to retrieve and display agent responses with citations

Let's get started! 🚀

---

## Step 1: Verify Required Python Packages

The dev container has already installed all the necessary Python packages for you:
- `azure-ai-projects`: The main SDK for Azure AI Foundry projects
- `azure-identity`: For authentication with Azure

Let's verify these packages are available and check their versions.

In [1]:
# Verify required packages are installed
import importlib.metadata

try:
    azure_ai_version = importlib.metadata.version('azure-ai-projects')
    azure_identity_version = importlib.metadata.version('azure-identity')
    
    print("✅ All required packages are installed!")
    print(f"📦 azure-ai-projects: {azure_ai_version}")
    print(f"📦 azure-identity: {azure_identity_version}")
except importlib.metadata.PackageNotFoundError as e:
    print(f"❌ Missing package: {e}")
    print("Please ensure the dev container has been properly set up.")

✅ All required packages are installed!
📦 azure-ai-projects: 1.0.0
📦 azure-identity: 1.25.1


## Step 2: Import Required Libraries

Now let's import all the libraries we'll need for this tutorial:
- `os`: For reading environment variables
- `AIProjectClient`: The main client for interacting with Azure AI Foundry
- `DefaultAzureCredential`: For Azure authentication
- `AzureAISearchTool`: Tool that enables the agent to search Azure AI Search indexes
- `ConnectionType`: For getting the Azure AI Search connection

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import AzureAISearchTool

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Step 3: Set Up Environment Variables

The following environment variables should already be configured in your `.env` file from the earlier setup steps. Let's just verify this here.

1. **AZURE_EXISTING_AIPROJECT_ENDPOINT**: Your Azure AI Foundry project endpoint
2. **AZURE_OPENAI_DEPLOYMENT**: The name of your deployed chat model 
3. **AZURE_AI_SEARCH_INDEX_NAME**: The name of your search index (should be "zava-products")



In [3]:
# Get environment variables
project_endpoint = os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT")
model_deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT") or os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
search_index_name = os.environ.get("AZURE_AI_SEARCH_INDEX_NAME") or os.environ.get("AZURE_SEARCH_INDEX_NAME")

# Verify that the variables are set
if not project_endpoint:
    raise ValueError("❌ AZURE_EXISTING_AIPROJECT_ENDPOINT environment variable is not set!")
if not model_deployment_name:
    raise ValueError("❌ AZURE_OPENAI_DEPLOYMENT environment variable is not set!")
if not search_index_name:
    raise ValueError("❌ AZURE_AI_SEARCH_INDEX_NAME environment variable is not set!")

print("✅ Environment variables loaded successfully!")
print(f"📍 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment_name}")
print(f"🔍 Search Index: {search_index_name}")

✅ Environment variables loaded successfully!
📍 Project Endpoint: https://aoai-4v6jkpb4qesje.services.ai.azure.com/api/projects/proj-4v6jkpb4qesje
🤖 Model Deployment: gpt-4.1
🔍 Search Index: zava-products


## Step 4: Verify Azure Credentials

- Before initializing the AI Project Client, let's verify that Azure credentials are available. 
- If not authenticated, you'll be prompted to run `az login`.

In [4]:
# Check Azure credentials
try:
    credential = DefaultAzureCredential()
    # Try to get a token to verify authentication
    token = credential.get_token("https://management.azure.com/.default")
    print("✅ Azure credentials verified successfully!")
except Exception as e:
    print("❌ Azure credentials not found or expired!")
    print("Please run 'az login' in the terminal to authenticate with Azure.")
    raise

# Create the AI Project Client
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

print("✅ AI Project Client initialized successfully!")

✅ Azure credentials verified successfully!
✅ AI Project Client initialized successfully!


## Step 5: Configure Azure AI Search Tool

Now we'll configure the Azure AI Search tool to connect to your search index. The tool:
- Gets the connection ID for your Azure AI Search service from your project
- Uses the "zava-products" index name from environment variables
- Retrieves the top 5 most relevant results for each search query

This allows the agent to search through product data and provide informed answers.

In [5]:
# Get the Azure AI Search connection from the project
# Include credentials to access the endpoint and key
azure_ai_search_connection = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    include_credentials=True
)
search_connection_id = azure_ai_search_connection.id

print(f"🔗 Azure AI Search Connection ID: {search_connection_id}")

# Create the Azure AI Search tool
ai_search_tool = AzureAISearchTool(
    index_connection_id=search_connection_id,
    index_name=search_index_name,
    top_k=5  # Retrieve the top 5 most relevant results
)

print("✅ Azure AI Search tool configured!")
print(f"🔍 Index Name: {search_index_name}")
print(f"📊 Top Results: 5")

🔗 Azure AI Search Connection ID: /subscriptions/7a880728-70d3-49d0-adde-4250716cfd94/resourceGroups/rg-nitya-ignite-PREL13/providers/Microsoft.CognitiveServices/accounts/aoai-4v6jkpb4qesje/projects/proj-4v6jkpb4qesje/connections/searchConnection
✅ Azure AI Search tool configured!
🔍 Index Name: zava-products
📊 Top Results: 5


### Test the Search Index (Optional)

Before creating an agent, let's verify the search index is working by performing a direct search query using the connection we just configured. This will help confirm that the index contains data and is accessible.

> **💡 Tip**: To verify the search index is working, try asking questions like:
> - "Tell me about Eggshell paint"
> - "What colors of paint are available?"
> - "Show me interior paint products"

In [6]:
# Test the search index with a sample query
from azure.search.documents import SearchClient

# Specify a test search query here
search_query = "Tell me about Eggshell paint"

# Get the search endpoint from environment variables
# This is more reliable than extracting from the Connection object
search_endpoint = os.environ.get("AZURE_AI_SEARCH_ENDPOINT") or os.environ.get("AZURE_SEARCH_ENDPOINT")

if not search_endpoint:
    raise ValueError("❌ AZURE_AI_SEARCH_ENDPOINT environment variable is not set!")

print(f"🔍 Testing search index with query: 'Eggshell paint'")
print(f"📍 Search Endpoint: {search_endpoint}")
print(f"📋 Index Name: {search_index_name}\n")

# Create a search client using DefaultAzureCredential (same as the rest of our setup)
# This is more secure than using API keys and aligns with Azure best practices
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=search_index_name,
    credential=credential  # Reusing the DefaultAzureCredential from Step 4
)

# Perform a simple search
try:
    results = search_client.search(search_text=search_query, top=3)
    
    print("✅ Search completed successfully!\n")
    print("="*80)
    print("📊 SEARCH RESULTS:")
    print("="*80 + "\n")
    
    result_count = 0
    for result in results:
        result_count += 1
        print(f"Result #{result_count}:")
        # Display key fields (adjust based on your index schema)
        for key, value in result.items():
            if key not in ['@search.score', '@search.rerankerScore'] and value:
                # Truncate long values
                str_value = str(value)
                if len(str_value) > 200:
                    str_value = str_value[:200] + "..."
                print(f"  {key}: {str_value}")
        print("-" * 80 + "\n")
    
    if result_count == 0:
        print("⚠️  No results found. The index might be empty or the query didn't match any documents.")
    else:
        print(f"✅ Found {result_count} result(s) - Search index is working correctly!")
        
except Exception as e:
    print(f"❌ Search failed: {e}")
    print("This might indicate an issue with the search index or credentials.")

🔍 Testing search index with query: 'Eggshell paint'
📍 Search Endpoint: https://srch-4v6jkpb4qesje.search.windows.net/
📋 Index Name: zava-products

✅ Search completed successfully!

📊 SEARCH RESULTS:

Result #1:
  content: Durable eggshell finish paint with subtle sheen, ideal for living rooms and bedrooms with easy cleanup.
  title: Interior Eggshell Paint
  stock: 80
  filepath: pfip000002
  price: 44.0
  id: PFIP000002
  url: /products/pfip000002
--------------------------------------------------------------------------------

Result #2:
  content: Disposable plastic liners that fit standard paint trays for quick cleanup and color changes.
  title: Paint Tray Liner Set
  filepath: pfpt000043
  price: 5.0
  id: PFPT000043
  url: /products/pfpt000043
--------------------------------------------------------------------------------

Result #3:
  content: Environmentally friendly zero-VOC paint for healthy indoor air quality in all living spaces.
  title: Zero VOC Interior Paint
  stock: 

## Step 6: Create Your AI Agent with Search Capabilities

Now we'll create an agent with the following properties:
- **model**: The AI model to use (e.g., GPT-4)
- **name**: A friendly name for your agent
- **instructions**: System instructions that define the agent's behavior as a product expert
- **tools**: The Azure AI Search tool for querying product data
- **tool_resources**: The search index configuration

The agent will be able to search the zava-products index and answer questions about products.

In [7]:
# Agent 1: Zava-Customer-Agent
# Create an agent with the Azure AI Search tool
agent = project_client.agents.create_agent(
    model=model_deployment_name,
    name="Zava-Customer-Agent",
    instructions="You are a polite and helpful shopping assistant for Zava products. Use the Azure AI Search tool to find information about products in the catalog and answer customer questions. Always cite your sources.",
    tools=ai_search_tool.definitions,
    tool_resources=ai_search_tool.resources
)

print("✅ Agent created successfully!")
print(f"🤖 Agent ID: {agent.id}")
print(f"📝 Agent Name: {agent.name}")
print(f"📋 Instructions: {agent.instructions}")

✅ Agent created successfully!
🤖 Agent ID: asst_jj1fls3VqnJfHxi9gfVPM8ix
📝 Agent Name: Zava-Customer-Agent
📋 Instructions: You are a polite and helpful shopping assistant for Zava products. Use the Azure AI Search tool to find information about products in the catalog and answer customer questions. Always cite your sources.


## Step 7: Create a Conversation Thread

A **thread** represents a conversation session between a user and the agent. It:
- Maintains conversation history
- Keeps context across multiple messages
- Can be reused for follow-up questions

Each thread has a unique ID that you'll use to add messages and run the agent.

In [8]:
# Create a thread for communication
thread = project_client.agents.threads.create()

print("✅ Conversation thread created!")
print(f"💬 Thread ID: {thread.id}")

✅ Conversation thread created!
💬 Thread ID: thread_zsMsbiRQHt0IHM9rZpVa9294


## Step 8: Ask Questions About Products

Now let's ask the agent questions about products in the zava-products index! We'll create a message with:
- **thread_id**: Which conversation thread to add the message to
- **role**: "user" (indicating this is from the user, not the agent)
- **content**: A question about products in the catalog

In this example, we're asking about sleeping bags to demonstrate the search capabilities.

In [9]:
# Add a message to the thread asking about products
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Tell me about your Eggshell paint products"
)

print("✅ User message added to thread!")
print(f"📨 Message ID: {message['id']}")
print(f"💭 Message content: {message['content']}")

✅ User message added to thread!
📨 Message ID: msg_uKWhUrMPI4de1B8jFiAvqRvg
💭 Message content: [{'type': 'text', 'text': {'value': 'Tell me about your Eggshell paint products', 'annotations': []}}]


## Step 9: Run the Agent

Now we'll tell the agent to process the user's message by creating and running a "run". The agent will:
- Search the zava-products index for relevant information
- Analyze the search results
- Generate a helpful response with product information
- Include citations to the source data

The `create_and_process` method automatically polls for completion.

In [10]:
# Create and process an agent run
print("🔄 Running the agent... (this may take a few moments)")

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)

print(f"✅ Run completed!")
print(f"📊 Run status: {run.status}")

# Check if the run failed
if run.status == "failed":
    print(f"❌ Run failed: {run.last_error}")
else:
    print("🎉 Agent successfully searched the product catalog!")

🔄 Running the agent... (this may take a few moments)
✅ Run completed!
📊 Run status: RunStatus.COMPLETED
🎉 Agent successfully searched the product catalog!


## Step 10: Retrieve and Display Messages with Citations

Now let's fetch all messages in the thread, which includes:
- The original user question
- The agent's response with product information
- Citations and references to the search results

The agent's response will include information from the zava-products index with source citations.

In [11]:
# Fetch all messages from the thread
from azure.ai.agents.models import MessageRole, ListSortOrder

messages = project_client.agents.messages.list(
    thread_id=thread.id,
    order=ListSortOrder.ASCENDING
)

print("\n" + "="*80)
print("📝 CONVERSATION HISTORY")
print("="*80 + "\n")

# Display each message
for message in messages:
    role_emoji = "👤" if message.role == MessageRole.USER else "🤖"
    print(f"{role_emoji} {message.role.upper()}:")
    
    # Display message content
    for content_item in message.content:
        if hasattr(content_item, 'text'):
            text = content_item.text.value
            print(f"{text}")
            
            # Display annotations (citations) if present
            if hasattr(content_item.text, 'annotations') and content_item.text.annotations:
                print("\n📚 Citations:")
                for annotation in content_item.text.annotations:
                    if hasattr(annotation, 'url_citation'):
                        citation = annotation.url_citation
                        print(f"  - {citation.title}: {citation.url}")
    
    print("-" * 80 + "\n")


📝 CONVERSATION HISTORY

👤 USER:
Tell me about your Eggshell paint products
--------------------------------------------------------------------------------

🤖 ASSISTANT:
Zava offers a range of Eggshell paint products, which are known for their low-sheen, durable, and washable finish—ideal for interior walls, woodwork, and sometimes metal. Eggshell paints typically strike a balance between matt and gloss, giving a subtle, elegant appearance while offering better washability than flat paints.

**Key features of Zava Eggshell paint products:**
- **Finish:** Soft, low-sheen eggshell (more durable than matt, less glossy than satin)
- **Usage:** Suitable for walls, ceilings, woodwork, and metal where a gentle sheen is desired
- **Durability:** Resistant to stains and easy to clean—ideal for living rooms, dining rooms, and hallways
- **Application:** Can be applied by brush, roller, or spray

**Popular Zava Eggshell Paint Options:**
- **Zava Premium Eggshell:** High coverage, available in a 

## Step 11: Try Another Question

Let's ask another question to demonstrate the agent's search capabilities. This time, we'll ask about a specific product feature.

In [12]:
# Ask another question
message2 = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Tell me about the Paint Tray Liner Set"
)

print(f"📨 Second question: {message2['content']}")

# Run the agent again
print("🔄 Running the agent...")
run2 = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)

print(f"✅ Run completed with status: {run2.status}")

# Display the latest response
messages = project_client.agents.messages.list(
    thread_id=thread.id,
    order=ListSortOrder.DESCENDING
)

print("\n" + "="*80)
print("🤖 LATEST AGENT RESPONSE:")
print("="*80 + "\n")

# Get the first message (most recent)
messages_list = list(messages)
if messages_list:
    latest_message = messages_list[0]
    if latest_message.role == MessageRole.AGENT:
        for content_item in latest_message.content:
            if hasattr(content_item, 'text'):
                print(content_item.text.value)
                
print("\n" + "="*80)

📨 Second question: [{'type': 'text', 'text': {'value': 'Tell me about the Paint Tray Liner Set', 'annotations': []}}]
🔄 Running the agent...
✅ Run completed with status: RunStatus.COMPLETED

🤖 LATEST AGENT RESPONSE:

The **Zava Paint Tray Liner Set** is a convenient accessory designed to simplify the painting process and make cleanup faster and easier. Here are the key details about this product:

- **Purpose:** Paint tray liners are disposable inserts used inside your paint tray. They protect the tray from messy paint residue, making it easy to switch between colors or clean up after painting.
- **Material:** Made from durable plastic that resists tearing and holds up well to both water-based and oil-based paints.
- **Compatibility:** Designed to fit standard-sized paint trays commonly used with rollers for walls, ceilings, and other large surfaces.
- **Convenience:** After painting, simply remove and discard the liner—no scrubbing necessary!
- **Pack Size:** Typically comes in a set 

## Step 12: Cleanup (Optional)

When you're done with the agent and thread, you can delete them to clean up resources. 

> **Note**: Uncomment the code below if you want to delete the agent and thread. Otherwise, they'll remain in your Azure AI Foundry project and you can continue to use them later.

In [13]:
# Delete the thread and agent to clean up resources
project_client.agents.threads.delete(thread.id)
print(f"🗑️  Deleted thread: {thread.id}")

project_client.agents.delete_agent(agent.id)
print(f"🗑️  Deleted agent: {agent.id}")


🗑️  Deleted thread: thread_zsMsbiRQHt0IHM9rZpVa9294
🗑️  Deleted agent: asst_jj1fls3VqnJfHxi9gfVPM8ix


## 🎉 Congratulations!

You've successfully completed the Azure AI Foundry Agents with Azure AI Search tutorial! Here's what you learned:

### Key Concepts Covered:
1. ✅ **Package Verification**: Confirmed the dev container has all required packages
2. ✅ **Authentication**: Verified Azure credentials and used DefaultAzureCredential
3. ✅ **Environment Setup**: Loaded project and search configuration using environment variables
4. ✅ **Client Initialization**: Created an AIProjectClient to connect to your project
5. ✅ **Azure AI Search Integration**: Configured the Azure AI Search tool with the zava-products index
6. ✅ **Agent Creation**: Built an AI agent with search capabilities
7. ✅ **Thread Management**: Created a conversation thread to maintain context
8. ✅ **Product Queries**: Asked questions about products in the search index
9. ✅ **Response Processing**: Retrieved agent responses with citations
10. ✅ **Multi-turn Conversation**: Had a multi-turn conversation with the agent
11. ✅ **Resource Cleanup**: Learned how to delete agents and threads


### Things You Can At Home:
- Experiment with different search queries
- Modify the agent's instructions for different use cases
- Explore combining Azure AI Search with other tools (like Code Interpreter)
- Adjust the `top_k` parameter to retrieve more or fewer results

### Resources:
- [Azure AI Foundry Documentation](https://learn.microsoft.com/azure/ai-foundry/)
- [Azure AI Foundry Agents Overview](https://learn.microsoft.com/azure/ai-foundry/agents/overview)
- [Azure AI Foundry Quickstart](https://learn.microsoft.com/azure/ai-foundry/quickstarts/get-started-code)
- [Azure AI Search Tool Samples (Python)](https://learn.microsoft.com/azure/ai-foundry/agents/how-to/tools/azure-ai-search-samples?pivots=python)
- [Azure AI Projects Python SDK Reference](https://learn.microsoft.com/python/api/overview/azure/ai-projects-readme)
- [Azure AI Search Documentation](https://learn.microsoft.com/azure/search/)
- [Python Samples for Azure AI Search](https://learn.microsoft.com/azure/search/samples-python)
- [Azure AI Projects Python Samples (GitHub)](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects/samples)

Happy building! 🚀